# Install Requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r "/content/drive/MyDrive/AI dr nesma" /content/data
!pip install ultralytics

In [34]:
import os, cv2
import shutil
import random
import numpy as np
from PIL import Image, ImageFilter
from ultralytics import YOLO
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
os.listdir('./data')

['Benign melanocytic nevus',
 'Melanoma',
 'Wart',
 'Herpes Zoster',
 'Tinea capitis',
 'Tinea circinata',
 'Psoraisis',
 'Normal skin',
 'Tinea Versicolor',
 'Vitiligo',
 'Basal cell carcinoma',
 'Alopecia areata']

## `Split data into training testing validation`

In [5]:
source_dir = "./data"
dest_dir = "./Data-medical"
classes = os.listdir(source_dir)

os.makedirs(os.path.join(dest_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "test"), exist_ok=True)

# === AUGMENTATION FUNCTIONS ===
def add_noise(img):
    noise = np.random.normal(0, 25, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)
    return noisy_img

def rotate_image(img, angle=15):
    h, w = img.shape[:2]
    matrix = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, matrix, (w, h))

def blur_image(img):
    return cv2.GaussianBlur(img, (5, 5), 0)

def apply_augmentations(img_path):
    img = cv2.imread(img_path)
    img_augmented = []
    img_augmented.append(img)
    img_augmented.append(add_noise(img))
    img_augmented.append(rotate_image(img, angle=random.choice([-15, 15])))
    img_augmented.append(blur_image(img))
    return img_augmented

# === ORGANIZE AND AUGMENT ===
for class_name in classes:
    os.makedirs(os.path.join(dest_dir, "train", class_name), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, "val", class_name), exist_ok=True)
    os.makedirs(os.path.join(dest_dir, "test", class_name), exist_ok=True)

    class_images = [f for f in os.listdir(os.path.join(source_dir, class_name))
                    if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    print(f"Organizing images for class: {class_name}...")
    train_files, temp_files = train_test_split(class_images, test_size=0.2, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)
    print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

    # === COPY + AUGMENT ===
    for idx, file in enumerate(train_files):
        src = os.path.join(source_dir, class_name, file)

        # Apply augmentations
        augmented_imgs = apply_augmentations(src)

        for i, aug_img in enumerate(augmented_imgs):
            dst_filename = f"{os.path.splitext(file)[0]}_aug{i}.jpg"
            dst = os.path.join(dest_dir, "train", class_name, dst_filename)
            cv2.imwrite(dst, aug_img)

    # Just copy val/test images without augmentation
    for file in val_files:
        src = os.path.join(source_dir, class_name, file)
        dst = os.path.join(dest_dir, "val", class_name, file)
        shutil.copy(src, dst)

    for file in test_files:
        src = os.path.join(source_dir, class_name, file)
        dst = os.path.join(dest_dir, "test", class_name, file)
        shutil.copy(src, dst)

print("✅ Dataset organized and augmented successfully!")

Organizing images for class: Benign melanocytic nevus...
Train: 190, Val: 24, Test: 24
Organizing images for class: Melanoma...
Train: 230, Val: 29, Test: 29
Organizing images for class: Wart...
Train: 180, Val: 22, Test: 23
Organizing images for class: Herpes Zoster...
Train: 180, Val: 22, Test: 23
Organizing images for class: Tinea capitis...
Train: 107, Val: 13, Test: 14
Organizing images for class: Tinea circinata...
Train: 135, Val: 17, Test: 17
Organizing images for class: Psoraisis...
Train: 199, Val: 25, Test: 25
Organizing images for class: Normal skin...
Train: 146, Val: 18, Test: 19
Organizing images for class: Tinea Versicolor...
Train: 177, Val: 22, Test: 23
Organizing images for class: Vitiligo...
Train: 254, Val: 32, Test: 32
Organizing images for class: Basal cell carcinoma...
Train: 159, Val: 20, Test: 20
Organizing images for class: Alopecia areata...
Train: 136, Val: 17, Test: 17
✅ Dataset organized and augmented successfully!


## `Train model`

In [6]:
model = YOLO('/content/new_model.pt')
results = model.train(data='./Data-medical', epochs=100)

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./Data-medical, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/new_model.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

100%|██████████| 5.35M/5.35M [00:00<00:00, 336MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3146.6±938.1 MB/s, size: 680.4 KB)


train: Scanning /content/Data-medical/train... 8364 images, 0 corrupt: 100%|██████████| 8364/8364 [00:02<00:00, 3446.33it/s]


train: New cache created: /content/Data-medical/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1762.7±2002.3 MB/s, size: 937.5 KB)


...

    100/100      6.89G    0.02376         12        224: 100%|██████████| 523/523 [00:56<00:00,  9.19it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:00<00:00, 35.69it/s]

                   all      0.912      0.996



100 epochs completed in 1.614 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train/weights/best.pt, 31.7MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,778,028 parameters, 0 gradients, 41.6 GFLOPs
train: /content/Data-medical/train... found 8364 images in 12 classes ✅ 
val: /content/Data-medical/val... found 261 images in 12 classes ✅ 
test: /content/Data-medical/test... found 266 images in 12 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:00<00:00, 11.62it/s]


                   all      0.939          1
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


## `Evaluation`

In [ ]:
# Evaluate on test set
metrics = model.val(data='./Data-medical', split='test')

In [8]:
print(f"Test Accuracy: {metrics.top1:.2f}%")
print(f"Test Accuracy (class): {metrics.top5:.2f}%")

Test Accuracy: 0.91%
Test Accuracy (class): 1.00%


In [35]:
def get_image_paths_and_labels(folder_path):
    image_paths = []
    labels = []
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(folder_path, class_name)
        if not os.path.exists(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            image_paths.append(os.path.join(class_dir, img_name))
            labels.append(class_name)
    return image_paths, labels

# Get test images and labels
test_folder = 'Data-medical/test'
test_image_paths, true_labels = get_image_paths_and_labels(test_folder)

# Get predictions
pred_labels = []
for img_path in test_image_paths:
    results = model(img_path)  # Predict on single image
    names_dict = results[0].names
    probs = results[0].probs.data.tolist()
    pred_labels.append(names_dict[np.argmax(probs)])

# Convert to numpy arrays
true_labels = np.array(true_labels)
pred_labels = np.array(pred_labels)

le = LabelEncoder()
le.fit(true_labels)  # Ensure all true classes are captured
true_labels = le.transform(true_labels)
pred_labels = le.transform(pred_labels)


image 1/1 /content/Data-medical/test/Benign melanocytic nevus/772.jpg: 224x224 Benign melanocytic nevus 1.00, Melanoma 0.00, Basal cell carcinoma 0.00, Alopecia areata 0.00, Wart 0.00, 5.9ms
Speed: 3.7ms preprocess, 5.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Data-medical/test/Benign melanocytic nevus/102.jpg: 224x224 Benign melanocytic nevus 1.00, Melanoma 0.00, Basal cell carcinoma 0.00, Tinea circinata 0.00, Tinea Versicolor 0.00, 4.4ms
Speed: 1.2ms preprocess, 4.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Data-medical/test/Benign melanocytic nevus/1288.jpg: 224x224 Benign melanocytic nevus 1.00, Melanoma 0.00, Tinea circinata 0.00, Basal cell carcinoma 0.00, Tinea Versicolor 0.00, 4.4ms
Speed: 1.2ms preprocess, 4.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Data-medical/test/Benign melanocytic nevus/2.jpg: 224x224 Benign melanocytic nevus 1.00, Me

In [36]:
# Classification report
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=classes, digits=4))


Classification Report:
                          precision    recall  f1-score   support

Benign melanocytic nevus     0.8889    0.9412    0.9143        17
                Melanoma     0.8500    0.8500    0.8500        20
                    Wart     0.8750    0.8750    0.8750        24
           Herpes Zoster     0.9200    1.0000    0.9583        23
           Tinea capitis     0.9310    0.9310    0.9310        29
         Tinea circinata     0.8571    0.9474    0.9000        19
               Psoraisis     0.9167    0.8800    0.8980        25
             Normal skin     0.9524    0.8696    0.9091        23
        Tinea Versicolor     0.8667    0.9286    0.8966        14
                Vitiligo     0.9375    0.8824    0.9091        17
    Basal cell carcinoma     0.9394    0.9688    0.9538        32
         Alopecia areata     0.9000    0.7826    0.8372        23

                accuracy                         0.9060       266
               macro avg     0.9029    0.9047    0

In [37]:
# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[16  0  0  0  0  0  0  0  1  0  0  0]
 [ 0 17  0  1  1  0  0  0  0  0  0  1]
 [ 0  1 21  0  1  0  0  0  0  0  0  1]
 [ 0  0  0 23  0  0  0  0  0  0  0  0]
 [ 0  0  2  0 27  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0  1  0]
 [ 0  0  0  1  0  0 22  1  0  1  0  0]
 [ 0  0  0  0  0  2  1 20  0  0  0  0]
 [ 1  0  0  0  0  0  0  0 13  0  0  0]
 [ 1  1  0  0  0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  1  0  0  0  0 31  0]
 [ 0  1  1  0  0  0  1  0  1  0  1 18]]


In [38]:
# Calculate per-class metrics
FP = cm.sum(axis=0) - np.diag(cm)  # False Positives
FN = cm.sum(axis=1) - np.diag(cm)  # False Negatives
TP = np.diag(cm)                   # True Positives

print("\nPer-class metrics:")
for i, class_name in enumerate(classes):
    precision = TP[i] / (TP[i] + FP[i]) if (TP[i] + FP[i]) > 0 else 0
    recall = TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"{class_name}:")
    print(f"  TP: {TP[i]}, FP: {FP[i]}, FN: {FN[i]}")
    print(f"  Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}\n")


Per-class metrics:
Benign melanocytic nevus:
  TP: 16, FP: 2, FN: 1
  Precision: 0.8889, Recall: 0.9412, F1: 0.9143

Melanoma:
  TP: 17, FP: 3, FN: 3
  Precision: 0.8500, Recall: 0.8500, F1: 0.8500

Wart:
  TP: 21, FP: 3, FN: 3
  Precision: 0.8750, Recall: 0.8750, F1: 0.8750

Herpes Zoster:
  TP: 23, FP: 2, FN: 0
  Precision: 0.9200, Recall: 1.0000, F1: 0.9583

Tinea capitis:
  TP: 27, FP: 2, FN: 2
  Precision: 0.9310, Recall: 0.9310, F1: 0.9310

Tinea circinata:
  TP: 18, FP: 3, FN: 1
  Precision: 0.8571, Recall: 0.9474, F1: 0.9000

Psoraisis:
  TP: 22, FP: 2, FN: 3
  Precision: 0.9167, Recall: 0.8800, F1: 0.8980

Normal skin:
  TP: 20, FP: 1, FN: 3
  Precision: 0.9524, Recall: 0.8696, F1: 0.9091

Tinea Versicolor:
  TP: 13, FP: 2, FN: 1
  Precision: 0.8667, Recall: 0.9286, F1: 0.8966

Vitiligo:
  TP: 15, FP: 1, FN: 2
  Precision: 0.9375, Recall: 0.8824, F1: 0.9091

Basal cell carcinoma:
  TP: 31, FP: 2, FN: 1
  Precision: 0.9394, Recall: 0.9688, F1: 0.9538

Alopecia areata:
  TP: 18

# Make Prediction For some Random Image

In [ ]:
model_predict = YOLO('models/new_model.pt')

In [ ]:
results = model_predict('D:\skin-Diseases-Detection-main\data\Tinea capitis\A-5-year-old-boy-with-a-large-kerion-celsi-before-treatment.png')

names_dict = results[0].names

probs = results[0].probs.data.tolist()

print(names_dict)
print(probs)

print(names_dict[np.argmax(probs)])